In [1]:
import boto3
import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
arg_date = '2022-12-31'
src_format = '%Y-%m-%d'
src_bucket = 'xetra-1234'
trg_bucket = 'udemy-xetra-1234'
columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
key = 'xetra_daily_report_' + datetime.today().strftime('%Y%m%d_%H%M%S') + '.parquet'

In [4]:
arg_date_dt = datetime.strptime(arg_date, src_format).date() - timedelta(days=1)

In [5]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(src_bucket)
objects = [obj for obj in bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], src_format).date() >= arg_date_dt ]

In [6]:
def csv_to_df(filename):
    csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
    data = StringIO(csv_obj)
    df = pd.read_csv(data, delimiter=',')
    return df
df_all = pd.concat([csv_to_df(obj.key) for obj in objects], ignore_index=True)

In [7]:
df_all = df_all.loc[:, columns]
df_all.dropna(inplace=True)

## Get opening perice per ISIN and day

In [8]:
df_all['opening_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('first')

## Get closing perice per ISIN and day

In [9]:
df_all['closing_price'] = df_all.sort_values(by=['Time']).groupby(['ISIN','Date'])['StartPrice'].transform('last')

## Aggregations

In [10]:
df_all = df_all.groupby(['ISIN','Date'], as_index=False).agg(opening_price_eur=('opening_price', 'min'), closing_price_eur=('closing_price', 'min'), 
                                             minimum_price_eur=('MinPrice', 'min'), maximum_price_eur=('MaxPrice', 'max'),
                                            daily_raded_volume=('TradedVolume', 'sum'))

## Percent Change Prev Closing

In [11]:
df_all['prev_closing_price'] = df_all.sort_values(by=['Date']).groupby(['ISIN'])['closing_price_eur'].shift(1)

In [12]:
df_all = df_all[df_all.Date == arg_date]

In [13]:
df_all['change_prev_closing_%'] = df_all['closing_price_eur'] - df_all['prev_closing_price'] / df_all['prev_closing_price'] * 100

In [14]:
df_all.drop(columns=['prev_closing_price'], inplace=True)

In [15]:
df_all = df_all.round(decimals=2)
df_all.head(3)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,change_prev_closing_%
1,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,-63.30
3,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,-91.43
5,AT0000606306,2022-12-31,14.51,15.00,13.65,15.28,107836,-85.00


## Write to S3

In [16]:
out_buffer = BytesIO()
df_all.to_parquet(out_buffer, index=False)
bucket_target = s3.Bucket(trg_bucket)
bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)

s3.Object(bucket_name='udemy-xetra-1234', key='xetra_daily_report_20241129_093737.parquet')

## Reading the uploaded file

In [17]:
for obj in bucket_target.objects.all():
    print(obj.key)

xetra_daily_report_20241127_232202.parquet
xetra_daily_report_20241128_175242.parquet
xetra_daily_report_20241128_185532.parquet
xetra_daily_report_20241129_093617.parquet
xetra_daily_report_20241129_093737.parquet


In [18]:
prq_obj = bucket_target.Object(key='xetra_daily_report_20241127_232202.parquet').get().get('Body').read()
data = BytesIO(prq_obj)
df_report = pd.read_parquet(data)

In [19]:
df_report.head(5)

,ISIN,Date,opening_price_eur,closing_price_eur,minimum_price_eur,maximum_price_eur,daily_raded_volume,change_prev_closing_%
0,AT000000STR1,2022-12-31,36.60,36.70,35.75,36.70,1773,-63.30
1,AT00000FACC2,2022-12-31,8.05,8.57,7.87,8.57,10205,-91.43
2,AT0000606306,2022-12-31,14.51,15.00,13.65,15.28,107836,-85.00
3,AT0000609607,2022-12-31,11.74,12.06,11.70,12.06,1065,-87.94
4,AT0000644505,2022-12-31,98.20,99.20,96.10,99.20,531,-0.80
